In [ ]:
import netket as nk
import numpy as np
import matplotlib.pyplot as plt

# Number of lattice points
L = 20

# Define custom graph
edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])

# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors)

# Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))

#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_color = [1, 2, 1, 2]

# Hilbert Space
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)

ahi = hi.all_states()
msr = np.ones(np.shape(ahi)[0])
for i in range(np.shape(ahi)[0]):
  for j in range(0, np.shape(ahi)[1], 2):
    if ahi[i,j] == 1:
      msr[i] = -msr[i]


def calsmsr(gvstate):
  smsr = 0
  absgstate = np.abs(gvstate)**2
  signstate = np.sign(gvstate)
  for i in range(np.shape(ahi)[0]):
    smsr= smsr + absgstate[i]*signstate[i]*msr[i]
  return smsr

def cals(gvstate, exact):
  s = 0
  absgstate = np.abs(exact)**2
  signstate = np.sign(exact)
  exparg = np.exp(np.angle(gvstate)*1j)
  for i in range(np.shape(ahi)[0]):
    s= s + absgstate[i]*signstate[i]*exparg[i]
  return s

# We need to specify the local operators as a matrix acting on a local Hilbert space
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

# Total spin operator
s_squared = nk.operator.LocalOperator(hi, dtype=complex)
sigsumx = nk.operator.LocalOperator(hi, dtype=complex)
sigsumy = nk.operator.LocalOperator(hi, dtype=complex)
sigsumz = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(L):
  sigsumx += nk.operator.spin.sigmax(hi,i)
for i in range(L):
  sigsumy += nk.operator.spin.sigmay(hi,i)
for i in range(L):
  sigsumz += nk.operator.spin.sigmaz(hi,i)
s_squared = sigsumx*sigsumx + sigsumy*sigsumy + sigsumz*sigsumz

#Define the RBMSymm
ma = nk.models.RBM(alpha=1, use_visible_bias=False, param_dtype=complex)

#Metropolis-Hastings sampler
sa = nk.sampler.MetropolisExchange(hilbert = hi, graph=g, d_max=2)

#Optimizer
op = nk.optimizer.Sgd(learning_rate=1e-3)
Sr = nk.optimizer.SR(diag_shift=1e-3,holomorphic=False)

N_samples=2000
N_iter=1000
#Define a variational state
vstate = nk.vqs.MCState(sampler=sa, model=ma, n_samples=N_samples)

energy = []
sf = []
s = []
smsr = []
corrzz = []
overlap = []
error = []
tspin = []
plotlist = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
import pandas as pd
df=pd.DataFrame(columns=["Fratio","Energy", "Sf", "Sign", "Smsr", "Overlap", "Error", "Tspin"])
import json
for i in plotlist:
    J = [1, i]
    bond_operator = [
        (J[0] * mszsz).tolist(),
        (J[1] * mszsz).tolist(),
        (J[0] * exchange).tolist(),
        (J[1] * exchange).tolist(),
    ]
    # Hamiltonian operator
    ha = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
    #Define a driver that performs VMC
    gs = nk.driver.VMC(ha, op, preconditioner=Sr, variational_state=vstate)

    path = '/content/drive/MyDrive/Colab Notebooks/NM/crbm/L_20/run1/'+str(N_samples)+'_'+str(N_iter)+'_'
    log = nk.logging.JsonLog(output_prefix=path + str(int(10*i)))

    #Run the optimization
    gs.run(n_iter=N_iter, out=log, obs={'Structure Factor': structure_factor, 'Total Spin': s_squared})

    # Exact Diagonalisation
    exact = nk.exact.lanczos_ed(ha, k=1, compute_eigenvectors=True)

    corr = []
    for r in range(1,int(L/2+1)):
      corrzzop = nk.operator.LocalOperator(hi, dtype=complex)
      for j in range(0, L):
        corrzzop += (nk.operator.spin.sigmaz(hi, j)*nk.operator.spin.sigmaz(hi, (j+r)%L))/L
      corr.append(vstate.expect(corrzzop).mean)

    data=json.load(open(path + str(int(10*i)) + '.log'))

    energy.append(np.mean((data['Energy']['Mean']['real'])[-10:]))
    sf.append(np.mean((data['Structure Factor']['Mean']['real'])[-10:]))
    gvstate = vstate.to_array(normalize=True)
    smsr.append(calsmsr(gvstate))
    s.append(cals(gvstate,exact[1][:,0]))
    corrzz.append(corr)
    overlap.append(np.abs(np.dot(np.conjugate(exact[1][:,0]),gvstate))**2)
    error.append(np.abs((energy[-1]-exact[0])/exact[0]))
    tspin.append(np.mean((data['Total Spin']['Mean']['real'])[-10:]))
    df.loc[len(df.index)]=[i,energy[-1],sf[-1],s[-1],smsr[-1],overlap[-1],error[-1],tspin[-1]]
    df.to_csv(path + '_vdata.csv')

corrzz = np.array(corrzz)

corrdf=pd.DataFrame(corrzz.T,columns=plotlist)
corrdf.to_csv(path + '_corrdata.csv')

# scorrzz = []
# for i in range(len(plotlist)):
#     corr = corrzz[i,:]
#     scorr = []
#     for k in np.arange(0, 2.1, 0.1):
#         scorrk = 0
#         for r in range(1,int(L/2+1)):
#             scorrk = scorrk + np.exp((np.pi*r*k)*1j)*corr[r-1]
#         scorr.append(scorrk)
#     scorrzz.append(scorr)

20
40
40


100%|██████████| 1000/1000 [13:08<00:00,  1.27it/s, Energy=-38.358-0.015j ± 0.077 [σ²=6.048, R̂=1.0087]]
